<h1>Compute 95% Confidence Interval for SIR Predictions</h1>

<p>In this tutorial we are going how to insert uncertainties in our preditctions. For achive it we are going to compute a non-parametric interval around our predictions. The interval is created using a techinique called bootstrap.</p>

<p>It's easy to compute confidence interval using the class SIR_EDO. This class now have a method calling fit_predictCI that returns a pandas data frame, with all you need to put uncertainties around your predictions. This methods take advantage of multprocessing system and can be distributed across your system.</p>

In [1]:
import multiprocessing as mp
import sys
#add the path were the models are
sys.path.append("../main/")
import pandas as pd
import numpy as np
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format
from itertools import repeat
import time
from functools import reduce
from modelos import SIR_GA
#set default theme for plts
theme_set(theme_linedraw())

In [2]:
data = pd.read_csv("../data/estados.csv")
#Select only Sao Paulo
sp = data[data["state"] == "SP"]
#Remove missing values to not crash the intervals
sp = sp.dropna()
#create a series with the cummulative number of cases
y = sp["TOTAL"]

#Give the number of days since the day of first case confirmed
x = range(0,len(sp["TOTAL"]))

In [3]:
#start model
model = SIR_GA(50000000)

In [4]:
model.fit(x = x, y = y)

In [5]:
model.predict(x)

array([1.00000000e+00, 1.44953603e+00, 2.00018507e+00, 2.67488538e+00,
       3.50421352e+00, 4.55049725e+00, 5.83162889e+00, 7.37132921e+00,
       9.30991205e+00, 1.16704311e+01, 1.45283100e+01, 1.81262933e+01,
       2.24827628e+01, 2.78076819e+01, 3.44846245e+01, 4.25217411e+01,
       5.24597681e+01, 6.48365628e+01, 7.96791775e+01, 9.82239854e+01,
       1.21141560e+02, 1.48590148e+02, 1.82628678e+02, 2.24097542e+02,
       2.75554483e+02, 3.38051863e+02, 4.15002532e+02, 5.09183587e+02,
       6.24585963e+02, 7.66104506e+02, 9.39659161e+02, 1.15200941e+03,
       1.41211571e+03, 1.72935702e+03, 2.11877167e+03, 2.59596621e+03])

In [ ]:
start = time.time()
resultdf = model.predictCI(x = x, y = y, ndays = 7, start = "2020-02-25",  bootstrap = 30, n_jobs = 8)
end = time.time()
print("Bootstrap tooks {} minutes to run".format((end - start)/60))

In [ ]:
resultdf

In [ ]:
#plot graph using ggplot
(ggplot(resultdf) + 
    geom_line(aes(x = "date", y = "Pred")) +
    geom_ribbon(aes(x = "date", ymin = "Pred_lb", ymax = "Pred_ub"), alpha = 0.5) +
    scale_x_datetime(breaks = date_breaks('5 days'), labels=date_format('%d-%b')) +
    ggtitle("Series of covid-19 for Sao Paulo") +
    ylab("Number of cases"))